<a href="https://colab.research.google.com/github/sheicky/ASR-Fine-Tunning/blob/main/moore_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning whisper for ASR

Installing packages

In [14]:
!pip install -U -q datasets[audio]  accelerate evaluate jiwer tensorboard
!pip install -U -q huggingface_hub
!pip install --upgrade transformers



Setting up connection with HF

In [15]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_Token')

In [16]:
from huggingface_hub import login

login(token=HF_TOKEN)

Loading the dataset

In [17]:
from datasets import load_dataset, DownloadConfig
goom_moore = load_dataset("sawadogosalif/MooreFRCollectionsAudios", split="train",download_config=DownloadConfig(token=HF_TOKEN))
print(goom_moore)

goom_moore = goom_moore.train_test_split(test_size=0.1, seed=42)

train_goom_moore = goom_moore["train"]
test_goom_moore = goom_moore["test"]

#goom_moore["train"] = load_dataset("sawadogosalif/MooreFRCollectionsAudios", split="train")
#goom_moore["test"] = load_dataset("sawadogosalif/MooreFRCollectionsAudios", split="test")
#goom_moore = load_dataset("sawadogosalif/MooreFRCollectionsAudios")


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'transcript', 'page', 'audio_sequence', 'duration'],
    num_rows: 14453
})


In [18]:
from datasets import DatasetDict


goom_moore = DatasetDict({
    "train": train_goom_moore,
    "test": test_goom_moore
})


In [19]:
print(goom_moore["train"])
print(goom_moore["test"])

Dataset({
    features: ['audio', 'transcript', 'page', 'audio_sequence', 'duration'],
    num_rows: 13007
})
Dataset({
    features: ['audio', 'transcript', 'page', 'audio_sequence', 'duration'],
    num_rows: 1446
})


In [20]:
goom_moore = goom_moore.remove_columns(["page","audio_sequence","duration"])
print(goom_moore)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcript'],
        num_rows: 13007
    })
    test: Dataset({
        features: ['audio', 'transcript'],
        num_rows: 1446
    })
})


Creating a WhisperProcessor

In [21]:
# loading the feature extractor
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [22]:
# Loading WhisperTokenizer

from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")

Preparing the Data

In [23]:
#Adapting the sampling rate
from datasets import Audio
goom_moore = goom_moore.cast_column("audio", Audio(sampling_rate=16000))

In [24]:
print(goom_moore["train"][0])
print(goom_moore["test"][0])

{'audio': {'path': 'segment_33.mp3', 'array': array([-0.01855782, -0.01423646, -0.01937903, ..., -0.04178075,
       -0.06378453, -0.11977633]), 'sampling_rate': 16000}, 'transcript': 'Saar-dãmb  ne wed-rʋʋdb me n tũ-ba'}
{'audio': {'path': 'segment_9.mp3', 'array': array([-0.01965364, -0.034068  , -0.0233672 , ..., -0.02222756,
       -0.02915639, -0.0371809 ]), 'sampling_rate': 16000}, 'transcript': 'Rẽ poore'}


In [25]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcript"]).input_ids
    return batch

In [26]:
goom_moore = goom_moore.map(prepare_dataset, remove_columns=goom_moore.column_names["train"], num_proc=2)

In [27]:
# goom_moore.save_to_disk("goom_moore_prepared")

In [28]:
# from datasets import load_from_disk

# goom_moore = load_from_disk("goom_moore_prepared")


Training and Evaluation

In [29]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")



In [30]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"transcript": feature["transcript"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [31]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small")

In [32]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

Evaluation Metrics

In [33]:
import evaluate

metric = evaluate.load("wer")

In [34]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Training Configuration

In [35]:
import transformers
print(transformers.__version__)


4.51.1


In [36]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-moore",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    #evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    #eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    #load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    remove_unused_columns=False
)

In [37]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_goom_moore ,
    eval_dataset=test_goom_moore,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

<ipython-input-37-ed2ed9af0c90>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [38]:
processor.save_pretrained(training_args.output_dir)

[]

# Training

In [39]:
trainer.train()

ValueError: You should supply an instance of `transformers.BatchFeature` or list of `transformers.BatchFeature` to this method that includes input_features, but you provided ['transcript']